In [65]:
import pandas as pd
import geopandas as gpd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import folium
from folium.plugins import MarkerCluster

In [ ]:
url = 'https://kfcku.com/store'

In [57]:
# Function to extract data from a store element
def extract_store_data(store):
    title = store.find('h4', class_='store--title').text.strip()
    address = store.find('div', class_='store--info').find('div').text.strip()
    phone = ""
    hours = ""
    if len(store.find_all('div', class_='store--info')) > 1:
        phone = store.find_all('div', class_='store--info')[1].text.strip()
    if len(store.find_all('div', class_='store--info')) > 2:
        hours = store.find_all('div', class_='store--info')[2].text.strip()

    latitude = float(store['data-lat'])
    longitude = float(store['data-lng'])

    return {
        'Title': title,
        'Address': address,
        'Phone': phone,
        'Hours': hours,
        'Latitude': latitude,
        'Longitude': longitude,
    }

# Set up the Selenium webdriver
driver = webdriver.Chrome()

# Open the webpage
url = 'https://kfcku.com/store'  # Replace with the actual URL
driver.get(url)

# Keep clicking the "More" button until it's not clickable
while True:
    try:
        more_button = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.CSS_SELECTOR, '.loadmore-store button'))
        )
        more_button.click()

        # Wait for the additional outlets to load
        WebDriverWait(driver, 10).until(
            EC.invisibility_of_element_located((By.CSS_SELECTOR, '.main-loader'))
        )

        # Introduce a delay of 1.5 seconds before the next click
        time.sleep(1.5)
    except Exception as e:
        print(f"Cannot click 'More' button: {e}")
        break

# Get the final page source after clicking "More" multiple times
final_page_source = driver.page_source

# Parse the final page source with BeautifulSoup
soup = BeautifulSoup(final_page_source, 'html.parser')

# Extract store data from each outlet element
store_elements = soup.find_all('div', class_='col-sm-6 col-lg-4')
data_list = [extract_store_data(store_element) for store_element in store_elements]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(data_list)

# Close the webdriver
driver.quit()

Cannot click 'More' button: Message: 



In [60]:
df

,Title,Address,Phone,Hours,Latitude,Longitude
0,ADAM MALIK MEDAN,"JL.ADAM MALIK NO. 5 / 7, Medan, Sumatera Utara",,,3.594566,98.668599
1,ADITYAWARMAN SURABAYA,"Jl. Adityawarman No. 371, Dukuh Pakis, Surabay...",,,-7.294636,112.731720
2,AEON JAKARTA GARDEN CITY,"Mall AEON Jakarta Garden City- Unit FC 3-11, K...",,,-6.172174,106.953081
3,AEON MALL,"Jl. BSD Raya Utama, Desa Sampora, Kec. Cisauk,...",,,-6.305178,106.641562
4,AHMAD YANI BANDUNG,"Jalan Ahmay Yani no 310, Kacapiring, Batunungg...",,,-6.913795,107.634276
...,...,...,...,...,...,...
728,WTC JAMBI,KOMPLEK WILLTOP BATANGHARI JL. SULTAN SAHA KEL...,,,-1.588571,103.615760
729,WTC SERPONG,"JL. RAYA SERPONG, GEDUNG WTC MATAHARI SERPONG...",,,-6.265934,106.655090
730,WTC SURABAYA,"Jl. Pemuda 27-31, Food Galery Lantai 3, Suraba...",,,-7.263227,112.747761
731,YOGYA SUKABUMI,"YOGYA DEPT. STORE JL. R.E MARTADINATA NO. 3, S...",,,-6.919923,106.928607


In [69]:
# Create a Folium map centered around Jakarta
map_center = [-6.2088, 106.8456]  # Jakarta's latitude and longitude
kfc_map = folium.Map(location=map_center, zoom_start=10)

# Create a MarkerCluster layer
marker_cluster = MarkerCluster().add_to(kfc_map)

# Add markers for each location to the MarkerCluster layer
for index, row in df.iterrows():
    folium.Marker([row['Latitude'], row['Longitude']], popup=row['Title']).add_to(marker_cluster)

In [70]:
kfc_map